In [ ]:
import os

import sys
import pprint

root = '/'
pprint.pprint(sys.path)

import_path = root + 'pigvae_all'
sys.path.append(import_path)
pprint.pprint(sys.path)

import_path2 = root + "ddpm-torch"
sys.path.append(import_path2)
pprint.pprint(sys.path)

['/opt/conda/lib/python310.zip',
 '/opt/conda/lib/python3.10',
 '/opt/conda/lib/python3.10/lib-dynload',
 '',
 '/opt/conda/lib/python3.10/site-packages']
['/opt/conda/lib/python310.zip',
 '/opt/conda/lib/python3.10',
 '/opt/conda/lib/python3.10/lib-dynload',
 '',
 '/opt/conda/lib/python3.10/site-packages',
 '/torch_cuda/pigvae_all']
['/opt/conda/lib/python310.zip',
 '/opt/conda/lib/python3.10',
 '/opt/conda/lib/python3.10/lib-dynload',
 '',
 '/opt/conda/lib/python3.10/site-packages',
 '/torch_cuda/pigvae_all',
 '/torch_cuda/ddpm-torch']


In [2]:
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data.distributed import DistributedSampler
import random
from torch_geometric.data import Data
from torch_geometric.utils import from_networkx
import networkx as nx
from networkx.algorithms.shortest_paths.dense import floyd_warshall_numpy

from networkx.generators.random_graphs import *
from networkx.generators.ego import ego_graph
from networkx.generators.geometric import random_geometric_graph

/opt/conda/lib/python3.10/site-packages/torch_geometric/typing.py:86: UserWarning: An issue occurred while importing 'torch-scatter'. Disabling its usage. Stacktrace: /opt/conda/lib/python3.10/site-packages/torch_scatter/_version_cpu.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings.warn(f"An issue occurred while importing 'torch-scatter'. "
/opt/conda/lib/python3.10/site-packages/torch_geometric/typing.py:97: UserWarning: An issue occurred while importing 'torch-cluster'. Disabling its usage. Stacktrace: /opt/conda/lib/python3.10/site-packages/torch_cluster/_version_cpu.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKSsb
  warnings.warn(f"An issue occurred while importing 'torch-cluster'. "
/opt/conda/lib/python3.10/site-packages/torch_geometric/typing.py:113: UserWarning: An issue occurred while importing 'torch-spline-conv'. Disabling its usage. Stacktrace: /opt/conda/lib/python3.10/site-packages/torch_spline_conv/_version_cpu.so: undefined symbol: _ZN

In [3]:
num_node_f = 36
num_edge_f = 6

In [4]:
hparams = {
    "vae":True,
    "kld_loss_scale":0.01,
    "perm_loss_scale":0.1,
    "property_loss_scale":0.5,
    "num_node_features":num_node_f,
    "num_edge_features":1+num_edge_f+1,
    "emb_dim": 50,
    'graph_encoder_hidden_dim': 256,
    'graph_encoder_k_dim': 64,
    'graph_encoder_v_dim': 64,
    'graph_encoder_num_heads': 16,
    'graph_encoder_ppf_hidden_dim': 512,
    'graph_encoder_num_layers': 16,
    'graph_decoder_hidden_dim': 256,
    'graph_decoder_k_dim': 64,
    'graph_decoder_v_dim': 64,
    'graph_decoder_num_heads': 16,
    'graph_decoder_ppf_hidden_dim': 512,
    'graph_decoder_num_layers': 16,
    "graph_decoder_pos_emb_dim": 64,
    'property_predictor_hidden_dim': 3,
    'num_properties': 1
}

In [5]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [6]:
from torch import nn
from torch.nn import Linear, LayerNorm, Dropout
from torch.nn.functional import relu, pad
from pigvae.graph_transformer import Transformer, PositionalEncoding
from pigvae.models import GraphAE

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions as distrib
import torch.distributions.transforms as transform

class Flow(transform.Transform, nn.Module):

    def __init__(self):
        transform.Transform.__init__(self)
        nn.Module.__init__(self)

    # Init all parameters
    def init_parameters(self):
        for param in self.parameters():
            param.data.uniform_(-0.01, 0.01)

    # Hacky hash bypass
    def __hash__(self):
        return nn.Module.__hash__(self)

class PlanarFlow(Flow):

    def __init__(self, dim):
        super(PlanarFlow, self).__init__()
        self.weight = nn.Parameter(torch.Tensor(1, dim))
        self.scale = nn.Parameter(torch.Tensor(1, dim))
        self.bias = nn.Parameter(torch.Tensor(1))
        self.init_parameters()

    def _call(self, z):
        f_z = F.linear(z, self.weight, self.bias)
        return z + self.scale * torch.tanh(f_z)

    def log_abs_det_jacobian(self, z):
        f_z = F.linear(z, self.weight, self.bias)
        psi = (1 - torch.tanh(f_z) ** 2) * self.weight
        det_grad = 1 + torch.mm(psi, self.scale.t())
        return torch.log(det_grad.abs() + 1e-9)

In [8]:
# Main class for normalizing flow
class NormalizingFlow(nn.Module):

    def __init__(self, dim, blocks, flow_length, density):
        super().__init__()
        biject = []
        for f in range(flow_length):
            for b_flow in blocks:
                biject.append(b_flow(dim))
        self.transforms = transform.ComposeTransform(biject)
        self.bijectors = nn.ModuleList(biject)
        self.base_density = density
        #self.final_density = distrib.TransformedDistribution(density, self.transforms)
        self.log_det = []

    def forward(self, z):
        self.log_det = []
        # Applies series of flows
        for b in range(len(self.bijectors)):
            self.log_det.append(self.bijectors[b].log_abs_det_jacobian(z))
            z = self.bijectors[b](z)

        return z, self.log_det

In [9]:
from pigvae.models import GraphEncoder, GraphDecoder, Permuter
from pigvae.models import BottleNeckEncoder, BottleNeckDecoder, PropertyPredictor

import torch
from torch.nn import Linear, LayerNorm, Dropout
from torch.nn.functional import relu, pad
from pigvae.graph_transformer import Transformer, PositionalEncoding
from pigvae.synthetic_graphs.data import DenseGraphBatch


import torch
from torch.nn import Linear, LayerNorm, Dropout
from torch.nn.functional import relu, pad
from pigvae.graph_transformer import Transformer, PositionalEncoding
#from pigvae.synthetic_graphs.data import DenseGraphBatch

block_planar = [PlanarFlow]

class GraphFlowVAE(torch.nn.Module):
    def __init__(self, hparams):
        super().__init__()
        self.vae = hparams["vae"]
        self.encoder = GraphEncoder(hparams)
        self.bottle_neck_encoder = BottleNeckEncoder(hparams)
        self.bottle_neck_decoder = BottleNeckDecoder(hparams)
        self.property_predictor = PropertyPredictor(hparams)
        self.permuter = Permuter(hparams)
        self.decoder = GraphDecoder(hparams)
        self.flow = NormalizingFlow(dim=hparams["emb_dim"], blocks=block_planar, flow_length=16, \
                               density=distrib.MultivariateNormal(torch.zeros(hparams["emb_dim"]), torch.eye(hparams["emb_dim"])))

    def encode(self, node_features, edge_features, mask):

        graph_emb, node_features = self.encoder(
            node_features=node_features,
            edge_features=edge_features,
            mask=mask,
        )
        graph_emb, mu, logvar = self.bottle_neck_encoder(graph_emb)

        return graph_emb, node_features, mu, logvar

    def decode(self, graph_emb, perm, mask=None):
        props = self.property_predictor(graph_emb).squeeze()

        graph_emb = self.bottle_neck_decoder(graph_emb)
        node_logits, edge_logits = self.decoder(
            graph_emb=graph_emb,
            perm=perm,
            mask=mask
        )

        return node_logits, edge_logits, props

    def forward(self, node_features, edge_features, mask, training, tau):

        #sampling z0 with VAE encoder
        z_0, node_features, mu, logvar = self.encode(node_features, edge_features, mask)
        perm = self.permuter(node_features, mask=mask, hard=not training, tau=tau)

        # sampling posterior  zk with flows
        z_k, list_ladj = self.flow(z_0)
        #Cross entropy ln p(z_k)
        log_p_zk = -0.5 * z_k * z_k
        #Entropy ln q(z_0)
        log_q_z0 = - torch.mean(torch.sum(logvar,dim=1)) * 1e-3
        #  ln q(z_0) - ln p(z_k)
        logs = (- log_p_zk).sum()
        # Add log determinants
        ladj = torch.cat(list_ladj)
        # ln q(z_0) - ln p(z_k) - sum[log det]
        logs -= torch.sum(ladj) #all flow loss

        nflow_loss = (logs / float(n_batch)) + log_q_z0
        graph_pred = self.decode(z_k, perm, mask)

        return graph_pred, perm, z_0, z_k, list_ladj, nflow_loss, mu, logvar


class BottleNeckEncoder(torch.nn.Module):
    def __init__(self, hparams):
        super().__init__()
        self.d_in = hparams["graph_encoder_hidden_dim"]
        self.d_out = hparams["emb_dim"]
        self.vae = hparams["vae"]
        if self.vae:
            self.w = Linear(self.d_in, 2 * self.d_out)
        else:
            self.w = Linear(self.d_in, self.d_out)

    def forward(self, x):
        x = self.w(relu(x))
        if self.vae:
            mu = x[:, :self.d_out]
            logvar = x[:, self.d_out:]
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            x = mu + eps * std
            return x, mu, logvar
        else:
            return x, None, None

In [10]:
model = GraphFlowVAE(hparams).to(device)

In [11]:
load_model_dir = root + 'save_models/qm9/pig-beta-e3nflow-vae_models/'

model = torch.load(load_model_dir + "pigvae_best_model.pt")
model.to(device).eval()

GraphFlowVAE(
  (encoder): GraphEncoder(
    (posiotional_embedding): PositionalEncoding()
    (graph_transformer): Transformer(
      (self_attn_layers): ModuleList(
        (0): SelfAttention(
          (w_qs): Linear(in_features=256, out_features=1024, bias=False)
          (w_ks): Linear(in_features=256, out_features=1024, bias=False)
          (w_vs): Linear(in_features=256, out_features=1024, bias=False)
          (fc): Linear(in_features=1024, out_features=256, bias=False)
          (attention): ScaledDotProductWithEdgeAttention(
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (dropout): Dropout(p=0.1, inplace=False)
          (layer_norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
        )
        (1): SelfAttention(
          (w_qs): Linear(in_features=256, out_features=1024, bias=False)
          (w_ks): Linear(in_features=256, out_features=1024, bias=False)
          (w_vs): Linear(in_features=256, out_features=1024, bias=False)
      

In [12]:
import os
import shutil
import argparse
import yaml
#from easydict import EasyDict
from tqdm.auto import tqdm
from glob import glob
import torch
#import torch.utils.tensorboard
from torch.nn.utils import clip_grad_norm_
from torch_geometric.data import DataLoader

# Loop

In [13]:
def load_pickes(load_dir):
    with open(load_dir + "node_features.pickle", 'br') as fa:
        node_features = pickle.load(fa)
        fa.close
          
    with open(load_dir + "edge_features.pickle", 'br') as fb:
        edge_features = pickle.load(fb)
        fb.close
          
    with open(load_dir + "masks.pickle", 'br') as fc:
        masks = pickle.load(fc)
        fc.close

    with open(load_dir + "props.pickle", 'br') as fd:
        props = pickle.load(fd)
        fd.close

    with open(load_dir + "all_targets.pickle", 'br') as fe:
        targets = pickle.load(fe)
        fe.close
        
    return node_features, edge_features, masks, props, targets

In [14]:
def graphs_to_embs(dataloader):
    
    z_mus = []
    z_vars= []
    zk_list = []
    t_list = []
    
    for batch_idx, batch_data in enumerate(dataloader):

        batch_stds = []
        
        if batch_idx % 10 == 0 :
            print(batch_idx)

        node_features, edge_features, mask, props, targets = batch_data
        node_features, edge_features, mask, props = node_features.to(device), edge_features.to(device), mask.to(device), props.to(device)
    
        z_0, node_features, mu, logvar = model.encode(node_features, edge_features, mask)
        z_k, list_ladj = model.flow(mu)
        batch_std = torch.exp(0.5 * logvar)
    
        for std in batch_std:
            diag_matrix = torch.diag(std * std)
            batch_stds.append(diag_matrix.cpu().detach().numpy())
        
        batch_stds = np.reshape(np.array(batch_stds), (-1, hparams["emb_dim"], hparams["emb_dim"]))

        z_vars.extend(batch_stds)
        z_mus.extend(mu.cpu().detach().numpy())
        zk_list.extend(z_k.cpu().detach().numpy())
        t_list.extend(targets.cpu().detach().numpy())
    
        del mu, logvar, batch_std, z_k
        torch.cuda.empty_cache()
        
    return np.array(z_mus), np.array(zk_list), np.array(t_list)

In [15]:
import pickle

load_dir = root + "dataset/eval_dataset/molecular_properties/qm9/e3graphs/"
sample_save_dir = load_model_dir + "samples_for_mcmc/"

print(sample_save_dir)

if not os.path.exists(sample_save_dir):
    os.makedirs(sample_save_dir)

node_features, edge_features, masks, props, targets = load_pickes(load_dir)
dataset = torch.utils.data.TensorDataset(node_features, edge_features, masks, props, targets)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=20, shuffle=False)

z_mus, z_ks, ys = graphs_to_embs(dataloader)
print(z_mus.shape, ys.shape)

np.save(sample_save_dir + "embs_mu.npy", z_mus)
np.save(sample_save_dir + "embs_zk.npy", z_ks)
np.save(sample_save_dir + "all_targets.npy", ys)

/torch_cuda/save_qm9-models/pig-beta-e3nflow-vae_models_50d/samples_for_mcmc/
0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
206